In [1]:
import pandas as pd
import numpy as np
import warnings
from datetime import datetime,timedelta
from pybaseball import batting_stats_range,pitching_stats_range,playerid_reverse_lookup
from bs4 import BeautifulSoup as soup
from urllib.request import Request, urlopen as Ureq
warnings.filterwarnings("ignore")

In [2]:
bat_df = pd.read_csv("Clean_Bat.csv",index_col = "Unnamed: 0")
pitch_df = pd.read_csv("Clean_Pitch.csv",index_col = "Unnamed: 0")
pitch_df = pitch_df[pitch_df['Date'] < "2023-06-06"]
bat_df = bat_df[bat_df['Date'] < "2023-06-06"]
pitch_df.tail()

,Name,Date,Opp,Team,Home_Away,GS,IP,H,R,ER,...,IBB,GDP,SF,SB,CS,PO,BF,GB/FB,LD,PU
47,Martín Pérez,2023-06-05,SLN,TEX,NaN,1,7.0,3,1,0,...,0,1,0,1,0,0,25,0.32,0.26,0.05
48,Nick Pivetta,2023-06-05,TBA,BOS,NaN,0,0.2,1,1,1,...,0,0,0,0,0,0,6,0.50,0.50,0.00
49,Colin Poche,2023-06-05,BOS,TBA,1,0,1.0,0,0,0,...,0,0,0,0,1,0,3,0.00,0.00,0.00
50,Trevor Richards,2023-06-05,HOU,TOR,NaN,0,1.0,2,2,2,...,0,0,0,0,0,0,5,0.00,0.33,0.00
44,Johan Oviedo,2023-06-05,OAK,PIT,NaN,1,7.0,4,3,2,...,0,0,1,1,0,0,29,0.37,0.42,0.00


In [3]:
start_date = "2023-06-06"
now = datetime.now()
end_date = now.strftime("%Y-%m-%d")
start_date = datetime.strptime(start_date,"%Y-%m-%d")
total_bat_df = pd.DataFrame()
total_pitch_df = pd.DataFrame()
now = now - timedelta(days = 1)
while start_date < now:
    next_day = start_date + timedelta(days=1)
    temp_bat_df = batting_stats_range(start_date.strftime("%Y-%m-%d"),)
    temp_pitch_df = pitching_stats_range(start_date.strftime("%Y-%m-%d"),)
    start_date = next_day
    data = playerid_reverse_lookup(temp_bat_df['mlbID'].unique(),key_type = "mlbam")
    last_list = []
    first_list = []
    for x in range(len(data)):
        last_list.append(data.iloc[x]['name_last'].capitalize())
        first_list.append(data.iloc[x]['name_first'].capitalize())
    data['name_last'] = last_list
    data['name_first'] = first_list
    data['Name'] = data['name_first'] + " " + data['name_last']
    key_dict = {data['key_mlbam'][i]: data['Name'][i] for i in range(len(data))}
    key_dict[663796] = "Coco Montes"
    temp_bat_df['mlbID'] = temp_bat_df['mlbID'].apply(lambda x: key_dict[x])
    temp_bat_df['Name'] = temp_bat_df['mlbID']

    names = temp_pitch_df['mlbID'].unique()
    int_names = [int(name) for name in names]
    data = playerid_reverse_lookup(int_names,key_type = "mlbam")
    last_list = []
    first_list = []
    for x in range(len(data)):
        last_list.append(data.iloc[x]['name_last'].capitalize())
        first_list.append(data.iloc[x]['name_first'].capitalize())
    data['name_last'] = last_list
    data['name_first'] = first_list
    data['Name'] = data['name_first'] + " " + data['name_last']
    data['key_mlbam'] = [str(name) for name in data['key_mlbam']]
    key_dict = {data['key_mlbam'][i]: data['Name'][i] for i in range(len(data))}
    key_dict['686654'] = "Ty Adcock"
    key_dict['682175'] = 'Joe Jacques'
    key_dict['676689'] = 'Ricky Karcher'
    key_dict['669391'] = 'Owen White'
    key_dict['676775'] = 'Keaton Winn'
    temp_pitch_df['mlbID'] = temp_pitch_df['mlbID'].apply(lambda x: key_dict[x])
    temp_pitch_df['Name'] = temp_pitch_df['mlbID']
    temp_bat_df['Team'] = temp_bat_df.Lev + temp_bat_df.Tm
    temp_pitch_df['Team'] = temp_pitch_df.Lev + temp_pitch_df.Tm
    team_dict = {'Maj-NLMilwaukee':"MIL",
                 'Maj-ALLos Angeles': "LAA",
                 'Maj-NLAtlanta': "ATL",
                 'Maj-NLArizona': "ARI",
                 'Maj-NLNew York': "NYN",
                 'Maj-ALHouston': "HOU",
                 'Maj-NLCincinnati':"CIN",
                 'Maj-NLSt. Louis': "SLN",
                 'Maj-NLSan Diego': "SDN",
                 'Maj-NLWashington': "WSN",
                 'Maj-ALKansas City': "KCR",
                 'Maj-ALCleveland': "CLE",
                 'Maj-NLPittsburgh':"PIT",
                 'Maj-NLChicago':"CHN", 
                 'Maj-ALChicago':"CHA",
                 'Maj-NLMiami':"MIA",
                 'Maj-ALOakland':"OAK",
                 'Maj-ALBoston':"BOS",
                 'Maj-ALTampa Bay':"TBA",
                 'Maj-ALMinnesota':"MIN",
                 'Maj-ALDetroit':"DET",
                 'Maj-NLSan Francisco':"SFN",
                 'Maj-NLLos Angeles':"LAN",
                 'Maj-ALToronto':"TOR",
                 'Maj-NLColorado':"COL",
                 'Maj-ALTexas':"TEX",
                 'Maj-NLPhiladelphia':"PHI",
                 'Maj-ALBaltimore':"BAL",
                 'Maj-ALSeattle':"SEA",
                 'Maj-ALNew York':"NYA"}
    temp_bat_df['Team'] = temp_bat_df['Team'].apply(lambda x: team_dict[x])
    temp_pitch_df['Team'] = temp_pitch_df['Team'].apply(lambda x: team_dict[x])
    temp_bat_df.drop(columns = ["Lev","Tm","mlbID"],inplace = True)
    temp_pitch_df.drop(columns = ["Lev","Tm","mlbID"],inplace = True)
    total_bat_df = pd.concat([total_bat_df,temp_bat_df])
    total_pitch_df = pd.concat([total_pitch_df,temp_pitch_df])
total_bat_df['Home_Away'] = total_bat_df[total_bat_df.columns[4]]
total_pitch_df['Home_Away'] = total_pitch_df[total_pitch_df.columns[4]]
bat_important_stats = ['Name','Date','Opp','Team','Home_Away','PA', 'AB', 'R', 'H', '2B', '3B', 'HR', 'RBI', 'BB', 'IBB', 'SO', 'HBP', 'SH', 'SF', 'GDP', 'SB', 'CS']
pitch_important_stats = ['Name','Date','Opp','Team','Home_Away','GS','IP', 'H', 'R', 'ER', 'BB', 'SO', 'HR', 'HBP', 'AB', '2B', '3B', 'IBB', 'GDP', 'SF', 'SB', 'CS', 'PO', 'BF', 'GB/FB', 'LD', 'PU']
total_bat_df = total_bat_df[bat_important_stats]
total_pitch_df = total_pitch_df[pitch_important_stats]
bat_df = pd.concat([bat_df,total_bat_df])
pitch_df = pd.concat([pitch_df,total_pitch_df])
bat_df['Home_Away'] = bat_df['Home_Away'].replace('@',1)
bat_df['Home_Away'] = bat_df['Home_Away'].replace('', 0) 
bat_df['Date'] = pd.to_datetime(bat_df['Date'])
pitch_df['Home_Away'] = pitch_df['Home_Away'].replace('@',1)
pitch_df['Home_Away'] = pitch_df['Home_Away'].replace('', 0) 
pitch_df['Date'] = pd.to_datetime(pitch_df['Date'])
pitch_df.to_csv("Clean_Pitch.csv")
bat_df.to_csv("Clean_Bat.csv")

IndexError: list index out of range

In [ ]:
## Use this cell to troubleshoot any callups that occur
## temp_pitch_df[temp_pitch_df['mlbID'] == "676775"]

In [49]:
team_dict = {'Milwaukee Brewers':"MIL",
             'Los Angeles Angels': "LAA",
             'Atlanta Braves': "ATL",
             "Arizona D'Backs": "ARI",
             'New York Mets': "NYN",
             'Houston Astros': "HOU",
             'Cincinnati Reds':"CIN",
             'St. Louis Cardinals': "SLN",
             'San Diego Padres': "SDN",
             'Washington Nationals': "WSN",
             'Kansas City Royals': "KCR",
             'Cleveland Guardians': "CLE",
             'Pittsburgh Pirates':"PIT",
             'Chicago Cubs':"CHN", 
             'Chicago White Sox':"CHA",
             'Miami Marlins':"MIA",
             'Oakland Athletics':"OAK",
             'Boston Red Sox':"BOS",
             'Tampa Bay Rays':"TBA",
             'Minnesota Twins':"MIN",
             'Detroit Tigers':"DET",
             'San Francisco Giants':"SFN",
             'Los Angeles Dodgers':"LAN",
             'Toronto Blue Jays':"TOR",
             'Colorado Rockies':"COL",
             'Texas Rangers':"TEX",
             'Philadelphia Phillies':"PHI",
             'Baltimore Orioles':"BAL",
             'Seattle Mariners':"SEA",
             'New York Yankees':"NYA"}
url = 'https://www.baseball-reference.com/leagues/MLB-schedule.shtml'
uClient = Ureq(url)
raw_content = uClient.read()
page_soup = soup(raw_content, "html.parser")
html = list(page_soup.children)[3]
games = str(html).split('h3')[3:]
end_date = datetime.today()
total_df = pd.read_csv("Previous_Games.csv",index_col = "Unnamed: 0")
y = len(total_df.Date.unique())*2
end_date = end_date - timedelta(days = 2)
try:
    date = datetime.strptime(games[y].split('>')[1].split('<')[0].split(', ')[1] + " " + games[0].split('>')[1].split('<')[0].split(', ')[2],"%B %d %Y")
except:
    print("Currently have all games played")
    date = end_date
while date < end_date:
    date = datetime.strptime(games[y].split('>')[1].split('<')[0].split(', ')[1] + " " + games[0].split('>')[1].split('<')[0].split(', ')[2],"%B %d %Y")
    y += 1
    x = 1
    while x < (len(games[y].split('.shtml">'))-3):
        temp_list = []
        temp_list.append(games[y].split('.shtml">')[x].split('<')[0])
        temp_list.append(games[y].split('.shtml">')[x].split('<')[1].split('(')[1].split(')')[0])
        temp_list.append(games[y].split('.shtml">')[x+1].split('<')[0])
        temp_list.append(games[y].split('.shtml">')[x+1].split('<')[1].split('(')[1].split(')')[0])
        temp_df = pd.DataFrame(temp_list).T
        new_date = date.strftime("%Y-%m-%d")
        temp_df['Date'] = new_date
        temp_df.columns = ['Visitor','V_Score','Home','H_Score','Date']
        temp_df['Visitor'] = temp_df['Visitor'].apply(lambda x: team_dict[x])
        temp_df['Home'] = temp_df['Home'].apply(lambda x: team_dict[x])
        total_df = pd.concat([total_df,temp_df])
        x+=3
    y += 1
total_df.to_csv("Previous_Games.csv")
total_df.head()

Currently have all games played


,Visitor,V_Score,Home,H_Score,Date
0,HOU,3,LAA,1,2022-04-07
0,SDN,2,ARI,4,2022-04-07
0,CIN,6,ATL,3,2022-04-07
0,MIL,4,CHN,5,2022-04-07
0,CLE,1,KCR,3,2022-04-07


In [52]:
total_df.tail()

,Visitor,V_Score,Home,H_Score,Date
0,NYA,7,NYN,6,2023-06-13
0,TBA,1,OAK,2,2023-06-13
0,CLE,3,SDN,6,2023-06-13
0,MIA,3,SEA,9,2023-06-13
0,SFN,11,SLN,3,2023-06-13


In [54]:
temp_pitch_df = pitching_stats_range("2023-05-07",)
temp_pitch_df

,Name,Age,#days,Lev,Date,Tm,,Opp,G,GS,...,StL,StS,GB/FB,LD,PU,WHIP,BAbip,SO9,SO/W,mlbID
1,Albert Abreu,27,37,Maj-AL,"May 7, 2023",New York,@,Tampa Bay,1,0,...,0.00,0.17,0.00,0.50,0.00,3.000,0.500,0.0,NaN,656061
2,Bryan Abreu,26,37,Maj-AL,"May 7, 2023",Houston,@,Seattle,1,0,...,0.33,0.20,NaN,NaN,NaN,0.000,NaN,27.0,NaN,650556
3,Jason Adam,31,37,Maj-AL,"May 7, 2023",Tampa Bay,,New York,1,0,...,0.24,0.12,0.25,0.00,0.00,1.000,0.250,0.0,NaN,592094
4,Sandy Alcantara,27,37,Maj-NL,"May 7, 2023",Miami,@,Chicago,1,1,...,0.12,0.12,0.52,0.30,0.13,1.080,0.409,9.7,NaN,645261
5,Tyler Alexander,28,37,Maj-AL,"May 7, 2023",Detroit,@,St. Louis,1,0,...,0.25,0.00,0.50,0.50,0.00,3.000,0.500,0.0,NaN,641302
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
146,Trevor Williams,31,37,Maj-NL,"May 7, 2023",Washington,@,Arizona,1,1,...,0.17,0.03,0.29,0.29,0.07,2.077,0.417,10.4,2.5,592866
147,Steven Wilson,28,37,Maj-NL,"May 7, 2023",San Diego,,Los Angeles,1,0,...,0.18,0.24,0.00,0.00,1.00,1.000,0.000,18.0,2.0,621051
148,Jimmy Yacabonis,31,37,Maj-NL,"May 7, 2023",New York,,Colorado,1,0,...,0.20,0.05,0.25,0.25,0.25,15.000,0.667,0.0,0.0,642231
149,Ryan Yarbrough,31,37,Maj-AL,"May 7, 2023",Kansas City,,Oakland,1,1,...,0.18,0.09,0.41,0.06,0.24,0.529,0.177,3.2,NaN,642232


In [55]:
temp_bat_df = batting_stats_range("2023-05-07",)
temp_bat_df

,Name,Age,#days,Lev,Date,Tm,,Opp,G,PA,...,SH,SF,GDP,SB,CS,BA,OBP,SLG,OPS,mlbID
1,CJ Abrams,22,37,Maj-NL,"May 7, 2023",Washington,@,Arizona,1,4,...,0,0,0,0,0,0.500,0.50,0.500,1.000,682928
2,Jos\xc3\xa9 Abreu,36,37,Maj-AL,"May 7, 2023",Houston,@,Seattle,1,4,...,0,0,1,0,0,0.250,0.25,0.500,0.750,547989
3,Ronald Acuna Jr.,25,37,Maj-NL,"May 7, 2023",Atlanta,,Baltimore,1,5,...,0,0,0,0,0,0.000,0.00,0.000,0.000,660670
4,Willy Adames,27,37,Maj-NL,"May 7, 2023",Milwaukee,@,San Francisco,1,5,...,0,1,0,0,0,0.667,0.60,1.667,2.267,642715
5,Jes\xc3\xbas Aguilar,33,37,Maj-AL,"May 7, 2023",Oakland,@,Kansas City,1,2,...,0,0,0,0,0,0.000,0.00,0.000,0.000,542583
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
324,Connor Wong,27,37,Maj-AL,"May 7, 2023",Boston,@,Philadelphia,1,3,...,0,0,0,0,0,0.000,0.00,0.000,0.000,657136
325,Austin Wynns,32,37,Maj-NL,"May 7, 2023",Colorado,@,New York,1,5,...,0,1,0,0,0,0.333,0.40,0.333,0.733,642851
326,Christian Yelich,31,37,Maj-NL,"May 7, 2023",Milwaukee,@,San Francisco,1,5,...,0,0,0,1,0,0.400,0.40,0.400,0.800,592885
327,Masataka Yoshida,29,37,Maj-AL,"May 7, 2023",Boston,@,Philadelphia,1,4,...,0,0,0,0,0,0.500,0.50,0.500,1.000,807799


In [56]:
start_date = "2023-06-06"
start_date

Timestamp('2022-07-24 00:00:00')